In [149]:
from typing import Dict, List, Callable, Literal, Optional
import enum
import heapq
import numpy as np

### Task 1

In [127]:
type Components = Dict[str, Literal[0, 1]]

class Environment:
    def __init__(self):
        self.__components: Components = \
            dict([(chr(ord('A') + i), np.random.randint(0, 2)) for i in range(9)])
        
    def display_state(self):
        print(self.__components)
    
    def get_percept(self) -> Components:
        return self.__components
    
    def patch(self, component: str):
        self.__components.update({component: 0})

class Agent:
    def __init__(self):
        self.__patches: List[str] = []
    
    def scan(self, components: Components):
        for component in components:
            if components[component] == 0:
                print(f'<success>: {component} is secure.')
                continue
            print(f'<warning>: {component} is not secure!')
            self.__patches.append(component)

    def act(self, patch: Callable[[str], None]):
        for component in self.__patches:
            patch(component)
        self.__patches.clear()

def run_agent():
    env = Environment()
    agent = Agent()

    env.display_state()
    agent.scan(env.get_percept())
    agent.act(env.patch)
    env.display_state()

run_agent()

{'A': 1, 'B': 1, 'C': 1, 'D': 1, 'E': 1, 'F': 0, 'G': 1, 'H': 1, 'I': 1}
<warning>: A is not secure!
<warning>: B is not secure!
<warning>: C is not secure!
<warning>: D is not secure!
<warning>: E is not secure!
<success>: F is secure.
<warning>: G is not secure!
<warning>: H is not secure!
<warning>: I is not secure!
{'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0}


### Task 2

In [128]:
class States(enum.Enum):
    UNDERLOADED = 'UNDERLOADED'
    BALANCED = 'BALANCED'
    OVERLOADED = 'OVERLOADED'

BALANCED_COUNT = 10

class Server:
    def __init__(self, initial_tasks: List[int] = []):
        self.__tasks: List[int] = []
        self.__state = States.UNDERLOADED
        for task in initial_tasks:
            self.add(task)
    
    def add(self, task: int):
        self.__tasks.append(task)
        self.__update_state()

    def remove(self) -> int:
        task: int = self.__tasks.pop()
        self.__update_state()
        return task

    def get_state(self) -> States:
        return self.__state

    def __str__(self):
        return f'status: {self.__state.value:<11} | tasks[{len(self.__tasks)}]: {', '.join(map(str, self.__tasks))}'
    
    def __gt__(self, obj):
        return len(self.__tasks) < len(obj.__tasks)
    
    def __len__(self):
        return len(self.__tasks)
    
    def __update_state(self):
        n = len(self.__tasks)
        if n > BALANCED_COUNT:
            self.__state = States.OVERLOADED
        elif n == BALANCED_COUNT:
            self.__state = States.BALANCED
        else:
            self.__state = States.UNDERLOADED

class Environment:
    def __init__(self):
        self.__servers: List[Server] = [
            Server(np.random.randint(0, 20, np.random.randint(15))) for _ in range(5)
        ]

    def display_state(self):
        total = 0
        for i, server in enumerate(self.__servers):
            print(f'{i}:', server)
            total += len(server)
        print('Total tasks:', total)
    
    def update_servers(self, servers: List[Server]):
        self.__servers = servers

    def get_precept(self) -> List[Server]:
        return self.__servers

class Agent:
    class ServerWrapper:
        def __init__(self, server: Server):
            self.server = server
        def __gt__(self, obj):
            return len(self.server) > len(obj.server)

    def __init__(self):
        self.__balanced: List[Server] = []
        self.__overloaded_pq: List[Server] = []
        self.__underloaded_pq: List[Agent.ServerWrapper] = []

    def scan(self, servers: List[Server]):
        for server in servers:
            if server.get_state() == States.OVERLOADED:
                heapq.heappush(self.__overloaded_pq, server)
            elif server.get_state() == States.UNDERLOADED:
                heapq.heappush(self.__underloaded_pq, Agent.ServerWrapper(server))
            else:
                self.__balanced.append(server)
    
    def act(self):
        while self.__overloaded_pq and self.__underloaded_pq:
            overloaded: Server = heapq.heappop(self.__overloaded_pq)
            underloaded: Agent.ServerWrapper = heapq.heappop(self.__underloaded_pq)

            while len(overloaded) > BALANCED_COUNT and len(underloaded.server) < BALANCED_COUNT:
                underloaded.server.add(overloaded.remove())

            if overloaded.get_state() != States.BALANCED:
                heapq.heappush(self.__overloaded_pq, overloaded)
            else:
                self.__balanced.append(overloaded)

            if underloaded.server.get_state() != States.BALANCED:
                heapq.heappush(self.__underloaded_pq, underloaded)
            else:
                self.__balanced.append(underloaded.server)
        
        return self.__balanced + self.__overloaded_pq + list(map(lambda s: s.server, self.__underloaded_pq))

def run_agent():
    env = Environment()
    agent = Agent()
    
    env.display_state()
    agent.scan(env.get_precept())
    env.update_servers(agent.act())
    env.display_state()

run_agent()

0: status: UNDERLOADED | tasks[6]: 11, 19, 16, 1, 18, 3
1: status: UNDERLOADED | tasks[3]: 2, 9, 0
2: status: BALANCED    | tasks[10]: 3, 4, 16, 16, 6, 4, 19, 3, 5, 0
3: status: UNDERLOADED | tasks[9]: 4, 19, 18, 12, 16, 12, 7, 14, 12
4: status: UNDERLOADED | tasks[8]: 11, 8, 18, 3, 11, 8, 19, 18
Total tasks: 36
0: status: BALANCED    | tasks[10]: 3, 4, 16, 16, 6, 4, 19, 3, 5, 0
1: status: UNDERLOADED | tasks[3]: 2, 9, 0
2: status: UNDERLOADED | tasks[6]: 11, 19, 16, 1, 18, 3
3: status: UNDERLOADED | tasks[9]: 4, 19, 18, 12, 16, 12, 7, 14, 12
4: status: UNDERLOADED | tasks[8]: 11, 8, 18, 3, 11, 8, 19, 18
Total tasks: 36


### Task 3

In [129]:
class Backup(enum.Enum):
  COMPLETED = 'Completed'
  FAILED = 'Failed'

choices = list(Backup)

class Environment:
  def __init__(self, backup_counts: int):
    self.__backups: List[Backup] = [np.random.choice(choices) for _ in range(backup_counts)]
  
  def display_state(self):
    print(*map(lambda obj: (obj[0], obj[1].value), enumerate(self.__backups)), sep='\n')

  def get_precept(self) -> List[Backup]:
    return self.__backups

  def retry_backup(self, idx: int):
    self.__backups[idx] = Backup.COMPLETED
  
class Agent:
  def __init__(self):
    self.__failed: List[int] = []
  
  def scan(self, backups: List[Backup]):
    self.__failed = [i for i, backup in enumerate(backups) if backup == Backup.FAILED]
    print(self.__failed)
  
  def act(self, retry_backup: Callable[[int], None]):
    for i in self.__failed: retry_backup(i)

def run_agent():
  env = Environment(10)
  agent = Agent()

  env.display_state()
  agent.scan(env.get_precept())
  agent.act(env.retry_backup)
  env.display_state()

run_agent()

(0, 'Completed')
(1, 'Completed')
(2, 'Completed')
(3, 'Failed')
(4, 'Completed')
(5, 'Failed')
(6, 'Failed')
(7, 'Completed')
(8, 'Completed')
(9, 'Failed')
[3, 5, 6, 9]
(0, 'Completed')
(1, 'Completed')
(2, 'Completed')
(3, 'Completed')
(4, 'Completed')
(5, 'Completed')
(6, 'Completed')
(7, 'Completed')
(8, 'Completed')
(9, 'Completed')


### Task 4

In [150]:
class Status(enum.Enum):
  SAFE = 'Safe'
  LOW = 'Low Risk Vulnerable'
  HIGH = 'High Risk Vulnerable'

choices = list(Status)

class Environment:
  def __init__(self):
    self.__components: Dict[str, Status] = {chr(ord('A') + i): np.random.choice(choices) for i in range(9)}
  
  def display_state(self):
    print(*map(lambda obj: f'{obj[0]}: {obj[1].value}', self.__components.items()), sep='\n')

  def get_precept(self) -> Dict[str, Status]:
    return self.__components
  
  def patch(self, component: str):
    self.__components.update({component: Status.SAFE})

class Agent:
  def __init__(self):
    self.__priority = {
      Status.SAFE: 0,
      Status.LOW: 3, # priority value
      Status.HIGH: 0
    }
    self.__vulnerable: Dict[str, Status] = {}

  def utility(self, status: Status):
    return self.__priority.get(status, 0)
  
  def scan(self, components: Dict[str, Status]):
    for component, status in components.items():
      if status != Status.SAFE:
        print(f'Warning: {component} is {status.value}')
        if status == Status.HIGH:
          print('Premium service needed to patch.')
        else:
          self.__vulnerable.update({component: status})
      else:
        print(f'Success: {component} is {status.value}')

  def choose_action(self) -> Optional[str]:
    max_component = max(self.__vulnerable, key=lambda key: self.utility(self.__vulnerable.get(key)), default=None)
    self.__vulnerable.pop(max_component, None)
    return max_component

def run_agent():
  env = Environment()
  agent = Agent()

  env.display_state()
  components = env.get_precept()
  agent.scan(components)
  while True:
    component = agent.choose_action()
    if not component:
      break
    env.patch(component)
  env.display_state()

run_agent()

A: High Risk Vulnerable
B: Low Risk Vulnerable
C: Safe
D: High Risk Vulnerable
E: Safe
F: High Risk Vulnerable
G: Low Risk Vulnerable
H: Low Risk Vulnerable
I: Safe
Premium service needed to patch.
Success: C is Safe
Premium service needed to patch.
Success: E is Safe
Premium service needed to patch.
Success: I is Safe
A: High Risk Vulnerable
B: Safe
C: Safe
D: High Risk Vulnerable
E: Safe
F: High Risk Vulnerable
G: Safe
H: Safe
I: Safe


### Task 5